In [1]:
#=======================================================================================
# P-TUNING NLI(Natural Language Interference:자연어 추론) 테스트 예제
#
# => input_ids : [CLS]senetence1(전제)[SEP]sentence2(가설)
# => attention_mask : 1111111111(전체,가설)0000000(그외)
# => token_type_ids : 0000000(전제)1111111(가설)00000000(그외)
# => laels : 참(수반:entailment), 거짓(모순:contradiction), 모름(중립:neutral)
#
#
# prefix-tuning => GPT-2, T5등 LM에서 접두사 prompt를 추가하여 훈련시키는 방식
#
# p-tuning => P-tuning은 prefix-tuning보다 유연 합니다. 
# 시작할 때뿐만 아니라 프롬프트 중간에 학습 가능한 토큰을 삽입하기 때문입니다. 
# https://github.com/THUDM/P-tuning
#
# p-tuing v2 => 새로운 방식이 아니라 NLU 향상을 위해, MLM 모델에 prefix-tuning을 적용한 방식
# https://github.com/THUDM/P-tuning-v2
#=======================================================================================

import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import os
from os import sys
sys.path.append('..')
from myutils import seed_everything, GPU_info, mlogging
from tqdm.notebook import tqdm


logger = mlogging(loggername="bertnlitest", logfilename="bertnlitest")
device = GPU_info()
seed_everything(111)

# 모델, tokenizer 경로 지정 
model_path = '../../data11/model/bert/bert-multilingual-cased-p-tuing-pp-nli-50'
vocab_path = '../../data11/model/bert/bert-multilingual-cased-p-tuing-pp-nli-50'

# p-tuningv2 prefixt 튜닝일때 eval 할 모델이 기존 훈련시 설정했던값과 동일하게 설정해야 함.
pre_seq_len = 50             # prefix 계수
prefix_projection = True     # True = two-layer MLP 사용함(Multi-layer perceptron(다중퍼셉트론))
prefix_hidden_size = 512     # prefix hidden size


logfilepath:bertnlitest_2022-07-13.log
True
device: cuda:0
cuda index: 0
gpu 개수: 1
graphic name: NVIDIA A30


In [2]:
#==============================================================================
# p-tuing v2 => 새로운 방식이 아니라 NLU 향상을 위해, MLM 모델에 prefix-tuning을 적용한 방식
# 참고 소스 : https://github.com/THUDM/P-tuning-v2
# 
# p-tuning-v2의 주요 기여는 원래 입력 전에 사용자 정의 길이의 레이어 프롬프트를 추가하고 
# 다운스트림 작업에 대한 후속 교육에서 BERT 모델의 모든 매개변수를 고정하고 이러한 프롬프트만 교육하는 것임.
# 설명 : https://zhuanlan.zhihu.com/p/459305102
#
# => P-tuning-v2의 구현 방식은 prefix N 시퀀스를 생성한 다음, 원래 bert 모델과 연결한다. 이때 bert의 past_key_values(*여기서는 decoding 속도 개선 목적이 아님)를 이용함
# => bert의 past_key_values로 prefix에대한 key 와 value 를 넘겨줘서, 기존 입력 key, value와 연결시키도록 함.
# => get_prompt() 함수 : prefix를 past_key_value 형식(batch_size, num_heads, sequence_length - 1, embed_size_per_head)으로 조정(만듬)
# => attention_mask : 기존 attention_mask +  prefix_attention_mask 
#==============================================================================
import copy
import torch
from torch._C import NoopLogger
import torch.nn
import torch.nn.functional as F
from torch import Tensor
from torch.nn import CrossEntropyLoss, MSELoss, BCEWithLogitsLoss
from transformers import BertModel, BertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutput, Seq2SeqLMOutput

# PrefixEncoder 클래스
class PrefixEncoder(torch.nn.Module):
    r'''
    The torch.nn model to encode the prefix
    Input shape: (batch-size, prefix-length)
    Output shape: (batch-size, prefix-length, 2*layers*hidden)
    '''
    def __init__(self, config):
        super().__init__()
        self.prefix_projection = config.prefix_projection
        if self.prefix_projection:
            # Use a two-layer MLP to encode the prefix
            self.embedding = torch.nn.Embedding(config.pre_seq_len, config.hidden_size)
            self.trans = torch.nn.Sequential(
                torch.nn.Linear(config.hidden_size, config.prefix_hidden_size),
                torch.nn.Tanh(),
                torch.nn.Linear(config.prefix_hidden_size, config.num_hidden_layers * 2 * config.hidden_size)
            )
        else:
            self.embedding = torch.nn.Embedding(config.pre_seq_len, config.num_hidden_layers * 2 * config.hidden_size)

    def forward(self, prefix: torch.Tensor):
        if self.prefix_projection:
            prefix_tokens = self.embedding(prefix)
            past_key_values = self.trans(prefix_tokens)
        else:
            past_key_values = self.embedding(prefix)
        return past_key_values
    
# BertPrefixForSequenceClassification 클래스    
class BertPrefixForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        self.bert = BertModel(config)
        self.dropout = torch.nn.Dropout(config.hidden_dropout_prob)
        self.classifier = torch.nn.Linear(config.hidden_size, config.num_labels)

        # ***bert 모델은 grad 업데이트 안함(역전파 끔)
        for param in self.bert.parameters():
            param.requires_grad = False
        
        self.pre_seq_len = config.pre_seq_len
        self.n_layer = config.num_hidden_layers  # 12
        self.n_head = config.num_attention_heads # 12
        self.n_embd = config.hidden_size // config.num_attention_heads  # 64 = 768 % 12

        self.prefix_tokens = torch.arange(self.pre_seq_len).long()
        self.prefix_encoder = PrefixEncoder(config)

        bert_param = 0
        for name, param in self.bert.named_parameters():
            bert_param += param.numel()
        all_param = 0
        for name, param in self.named_parameters():
            all_param += param.numel()
        total_param = all_param - bert_param
        print('total param is {}'.format(total_param)) # 9860105
    
    def get_prompt(self, batch_size):
        # [32,20] 만듬 (32:batch_size, 20 은 self.prefix_tokens arange로 생성한 값)
        prefix_tokens = self.prefix_tokens.unsqueeze(0).expand(batch_size, -1).to(self.bert.device)
        #print(f'prefix_tokens=>{prefix_tokens.shape}')
        
        past_key_values = self.prefix_encoder(prefix_tokens)
        #print(f'past_key_values1=>{past_key_values.shape}')
            
        # bsz, seqlen, _ = past_key_values.shape
        past_key_values = past_key_values.view(
            batch_size,
            self.pre_seq_len,
            self.n_layer * 2, 
            self.n_head,
            self.n_embd
        )
        # batch_size:32, pre_seq_len:20, nlayer:12, nhead:12, n_emb:64
        #print(f'*batch_size:{batch_size}, pre_seq_len:{self.pre_seq_len}, nlayer:{self.n_layer}, nhead:{self.n_head}, n_emb:{self.n_embd}') 
        
        #print(f'past_key_values2=>{past_key_values.shape}')  # torch.Size([32, 20, 24, 12, 64])
            
        past_key_values = self.dropout(past_key_values)
        #print(f'past_key_values3=>{past_key_values.shape}')  # torch.Size([32, 20, 24, 12, 64])
            
        past_key_values = past_key_values.permute([2, 0, 3, 1, 4]).split(2)
        # torch.Size([2, 32, 12, 20, 64]), len"12
        #print(f'past_key_values4=>{past_key_values[0].shape}, len"{len(past_key_values)}')
        
        return past_key_values

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        #print(f'return_dict=>{return_dict}') # True
            
        batch_size = input_ids.shape[0]
        
        # past_key_values를 지정하면, 어텐션 블록에서, 미리 계산된 키 및 값 숨겨진 상태를 포함한다.
        # 즉 p-tuning-v2는 추가된 prefix value 와 key를 기존 입력 value와 key를 서로 연결시키는 구조이다.
        # 따라서 prefix의 past_key_values를 구하는데 (batch_size, num_heads, sequence_length - 1, embed_size_per_head) 형태로 구하면 된다.
        # 참고 : https://zhuanlan.zhihu.com/p/459305102
        # => past_key_values 를 지정하지 않으면 아래와 같은 에러가 발생함.
        # RuntimeError: The size of tensor a (128) must match the size of tensor b (148) at non-singleton dimension 3
        past_key_values = self.get_prompt(batch_size=batch_size)
        
        # attention_mask : 기존 attention_mask +  prefix_attention_mask 더함.
        # prefix_attention_mask 를 구하고, 
        prefix_attention_mask = torch.ones(batch_size, self.pre_seq_len).to(self.bert.device)
        #print(f'prefix_attention_mask=>{prefix_attention_mask.shape}') # torch.Size([32, 20])
        
        # 기존 attention_mask +  prefix_attention_mask 합침.
        attention_mask = torch.cat((prefix_attention_mask, attention_mask), dim=1) # 128(기존 attention_mask) + 20(prefix_attention_mask) = 148     
        #print(f'*attention_mask2=>{attention_mask.shape}') # torch.Size([32, 148])
        #print(f'*input_ids=>{input_ids.shape}')
        #print(f'*token_type_ids=>{token_type_ids.shape}')
        #print(f'*past_key_values=>{past_key_values[0].shape}')
        
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
            past_key_values=past_key_values,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze())
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [3]:
# tokeniaer 및 model 설정
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = BertTokenizer.from_pretrained(vocab_path, do_lower_case=False)
# strip_accents=False : True로 하면, 가자 => ㄱ ㅏ ㅈ ㅏ 식으로 토큰화 되어 버림(*따라서 한국어에서는 반드시 False)
# do_lower_case=False : # 소문자 입력 사용 안함(한국어에서는 반드시 False)
#tokenizer = BertTokenizer(vocab_file=vocab_path, strip_accents=False, do_lower_case=False) 


# tokenier 테스트
print(len(tokenizer))
print(tokenizer.encode("눈에 보이는 반전이었지만 영화의 흡인력은 사라지지 않았다", "정말 재미있다"))
print(tokenizer.convert_ids_to_tokens(131027))
print(tokenizer.convert_tokens_to_ids('정말'))

119547
[101, 9034, 10530, 9356, 31728, 9321, 16617, 10739, 69708, 42428, 10459, 10020, 12030, 28143, 10892, 9405, 17342, 12508, 12508, 49137, 102, 9670, 89523, 9659, 22458, 76820, 102]
[UNK]
100


In [4]:
# 평가 data loader 생성
from os import sys
sys.path.append('..')
from myutils import ClassificationDataset, KlueNLICorpus, data_collator
from torch.utils.data import DataLoader, RandomSampler

#############################################################################
# 변수 설정
#############################################################################
max_seq_len = 128   # 글자 최대 토큰 길이 해당 토큰 길이 이상은 잘린다.
batch_size = 32        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = True   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
#############################################################################

# corpus 파일 설정
corpus = KlueNLICorpus()

# 평가 dataset 생성
file_fpath = '../../data11/korpora/klue-nli/klue-nli-v1.1_dev.json'
dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)

# 평가 dataloader 생성
eval_loader = DataLoader(dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)

print('eval_loader_len: {}'.format(len(eval_loader)))

Creating features from dataset file at ../../data11/korpora/klue-nli/klue-nli-v1.1_dev.json
loading data... LOOKING AT ../../data11/korpora/klue-nli/klue-nli-v1.1_dev.json
tokenize sentences, it could take a lot of time...
tokenize sentences [took %.3f s] 0.9729437828063965


  0%|          | 0/3000 [00:00<?, ?it/s]

*** Example ***
sentence A, B: 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다. + 어떤 방에서도 흡연은 금지됩니다.
tokens: [CLS] 흡 ##연 ##자 ##분 ##들은 발 ##코 ##니 ##가 있는 방 ##이 ##면 발 ##코 ##니 ##에서 흡 ##연 ##이 가 ##능 ##합 ##니다 . [SEP] 어떤 방 ##에서 ##도 흡 ##연 ##은 [UNK] . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
label: contradiction
features: ClassificationFeatures(input_ids=[101, 10020, 25486, 13764, 37712, 22879, 9323, 25517, 25503, 11287, 13767, 9328, 10739, 14867, 9323, 25517, 25503, 11489, 10020, 25486, 10739, 8843, 74986, 

In [5]:
from transformers import AutoConfig

corpus_labels = corpus.get_labels()

label2id = {l: i for i, l in enumerate(corpus_labels)}
id2label = {id: label for label, id in label2id.items()}
print(f"label2id : {label2id}")
print(f"id2label : {id2label}")

config = AutoConfig.from_pretrained(
    model_path,
    num_labels=3,  #nli이므로 3으로 함
    #label2id=dataset.label2id,
    #id2label=dataset.id2label,
    label2id=label2id,
    id2label=id2label,
    finetuning_task="rte",
    revision="main"
)

#========================================================================
# 훈련 모델에 따라 아래값들을 바꿔줘야 함.
#========================================================================

#get_model 에서 --prefix인경우 config 인자 설정해 주고 있음.
config.hidden_dropout_prob = 0.1
config.pre_seq_len = pre_seq_len             # prefix 계수
config.prefix_projection = prefix_projection    # True = two-layer MLP 사용함(Multi-layer perceptron(다중퍼셉트론))
config.prefix_hidden_size = prefix_hidden_size     # prefix hidden size
#========================================================================

print(config.num_hidden_layers)
print(config.num_attention_heads)
print(config.hidden_size)

label2id : {'entailment': 0, 'contradiction': 1, 'neutral': 2}
id2label : {0: 'entailment', 1: 'contradiction', 2: 'neutral'}
12
12
768


In [6]:
#############################################################################################
# 변수들 설정
# - model_path : from_pretrained() 로 호출하는 경우에는 모델파일이 있는 폴더 경로나 
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')
#          torch.load(model)로 로딩하는 경우에는 모델 파일 풀 경로
#
# - vocab_path : from_pretrained() 호출하는 경우에는 모델파일이 있는 폴더 경로나
#          huggingface에 등록된 모델명(예:'bert-base-multilingual-cased')   
#          BertTokenizer() 로 호출하는 경우에는 vocab.txt 파일 풀 경로,
#############################################################################################
     
# p-tuning 모델인 경우
model = BertPrefixForSequenceClassification.from_pretrained(model_path, config=config, revision="main")

# 레벨을 1개만 선택하는 경우
#model = BertForSequenceClassification.from_pretrained(model_path, num_labels=3)
#model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=6)

# 레벨을 멀티로 선택해야 하는 경우
#model = BertForSequenceClassification.from_pretrained(model_path, problem_type="multi_label_classification",num_labels=6)

model.to(device)

total param is 9890051


BertPrefixForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [7]:
logger.info(f"=== model: {model_path} ===")
logger.info(f"num_parameters: {model.num_parameters()}")

2022-07-13 12:50:02,102 - bertnlitest - INFO - === model: ../../data11/model/bert/bert-multilingual-cased-p-tuing-pp-nli-50 ===
2022-07-13 12:50:02,105 - bertnlitest - INFO - num_parameters: 187743491


In [8]:
import time

# 평가 시작
model.eval()

total_loss = 0
total_len = 0
total_correct = 0

start = time.time()
logger.info(f'---------------------------------------------------------')

for data in tqdm(eval_loader):
     # 입력 값 설정
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)       
    labels = data['labels'].to(device)
 
    # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
    # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
    # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
    with torch.no_grad():
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       labels=labels)
    
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits
        #print(F.softmax(logits))
        #pred = torch.argmax(F.softmax(logits), dim=1)
        pred = torch.argmax(logits, dim=1)
        #print(logits)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)

logger.info(f"eval-accuracy: {total_correct / total_len}")
logger.info(f'---------------------------------------------------------')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-07-13 12:50:02,162 - bertnlitest - INFO - ---------------------------------------------------------


  0%|          | 0/94 [00:00<?, ?it/s]

2022-07-13 12:50:07,422 - bertnlitest - INFO - eval-accuracy: 0.6893333333333334
2022-07-13 12:50:07,424 - bertnlitest - INFO - ---------------------------------------------------------
2022-07-13 12:50:07,425 - bertnlitest - INFO - === 처리시간: 5.263 초 ===
2022-07-13 12:50:07,426 - bertnlitest - INFO - -END-

